In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, BaseMessage
from typing import TypedDict, Annotated, Literal
from pydantic import Field, BaseModel
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import InMemorySaver
import time

load_dotenv()


True

In [3]:
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str
    step3: str


In [4]:
def step_1(state: CrashState) -> CrashState:
    print("Executing Step 1")
    return {"step1": "done", "input": state["input"]}


def step_2(state: CrashState) -> CrashState:
    print("Executing Step 2")
    # Simulate a crash
    time.sleep(30)
    return {"step2": "done"}


def step_3(state: CrashState) -> CrashState:
    print("Executing Step 3")
    return {"step3": "done"}

In [5]:
graph = StateGraph(CrashState)

graph.add_node("step_1", step_1)
graph.add_node("step_2", step_2)
graph.add_node("step_3", step_3)

graph.add_edge(START, "step_1")
graph.add_edge("step_1", "step_2")
graph.add_edge("step_2", "step_3")
graph.add_edge("step_3", END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [7]:
workflow.invoke(
    {"input": "start"}, config={"configurable": {"thread_id": "crash_test"}}
)

Executing Step 1
Executing Step 2


KeyboardInterrupt: 

In [8]:
workflow.get_state_history({"configurable": {"thread_id": "crash_test"}})

<generator object Pregel.get_state_history at 0x0000022E1F4489E0>